In [1]:
import scrapy
import requests
from scrapy.http import TextResponse 

In [2]:
url = 'http://corners.gmarket.co.kr/Bestsellers'

In [3]:
req = requests.get(url)
response = TextResponse(req.url, body = req.text, encoding = 'utf-8')

In [4]:
links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
len(links)

200

In [5]:
links[0]

'http://item.gmarket.co.kr/Item?goodscode=1840147374&ver=637302513750104238'

In [6]:
req = requests.get(links[1])
response = TextResponse(req.url, body = req.text, encoding = 'utf-8')

title = response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract()
o_price = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(",", "")
s_price = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",", "")
discount_rate  = str(round(1-int(s_price) / int(o_price)*100 , 2)) + "%"
title, o_price, s_price, discount_rate

('[동서] 오레오웨하스150g x3개입+간식쟁반 ', '29800', '11900', '-38.93%')

In [7]:
!scrapy startproject crawler

Error: scrapy.cfg already exists in /Users/hyunjoon/Documents/dss/programming/05_crawling/crawler


In [8]:
!tree crawler

crawler
├── crawler
│   ├── __init__.py
│   ├── __pycache__
│   │   ├── __init__.cpython-37.pyc
│   │   ├── items.cpython-37.pyc
│   │   └── settings.cpython-37.pyc
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       ├── __pycache__
│       │   ├── __init__.cpython-37.pyc
│       │   └── spider.cpython-37.pyc
│       └── spider.py
└── scrapy.cfg

4 directories, 13 files


In [9]:
import scrapy

In [10]:
!cat crawler/crawler/items.py

import scrapy

class CrawlerItem(scrapy.Item):

    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    discount_rate = scrapy.Field()
    link = scrapy.Field()


In [11]:
%%writefile crawler/crawler/items.py
import scrapy

class CrawlerItem(scrapy.Item):

    title = scrapy.Field()
    s_price = scrapy.Field()
    o_price = scrapy.Field()
    discount_rate = scrapy.Field()
    link = scrapy.Field()

Overwriting crawler/crawler/items.py


In [12]:
%%writefile crawler/crawler/spiders/spider.py
import scrapy
from crawler.items import CrawlerItem

class Spider(scrapy.Spider):
    name = "GmarketBestsellers"
    allow_domain = ["gmarket.co.kr"]
    start_urls = ["http://corners.gmarket.co.kr/Bestsellers"]
    
    def parse(self, response): 
        links = response.xpath('//*[@id="gBestWrap"]/div/div[3]/div[2]/ul/li/div[1]/a/@href').extract()
        for link in links:
            yield scrapy.Request(link, callback = self.page_content)

    def page_content(self, response):
        item = CrawlerItem()
        item["title"] =response.xpath('//*[@id="itemcase_basic"]/h1/text()')[0].extract()
        item["s_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/strong/text()')[0].extract().replace(",", "")
        try:
            item["o_price"] = response.xpath('//*[@id="itemcase_basic"]/p/span/span/text()')[0].extract().replace(",", "")
        except:
            item["o_price"] =  item["s_price"]
        item["discount_rate"]  = str(round(1-int(item["s_price"] ) / int(item["o_price"])*100 , 2)) + "%"
        item["link"] = response.url
        yield item

Overwriting crawler/crawler/spiders/spider.py


In [17]:
%%writefile run.sh
cd crawler
scrapy crawl GmarketBestsellers -o GmarketBestsellers.csv

Overwriting run.sh


In [ ]:
!chmod +x run.sh

In [20]:
! ./run.sh

2020-07-13 15:41:10 [scrapy.utils.log] INFO: Scrapy 2.2.0 started (bot: crawler)
2020-07-13 15:41:10 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-18.7.0-x86_64-i386-64bit
2020-07-13 15:41:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-13 15:41:10 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'crawler',
 'NEWSPIDER_MODULE': 'crawler.spiders',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['crawler.spiders']}
2020-07-13 15:41:10 [scrapy.extensions.telnet] INFO: Telnet Password: a7653a43e90c567f
2020-07-13 15:41:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',

2020-07-13 15:41:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1101169778&ver=637302516704398374>
{'discount_rate': '-68.88%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1101169778&ver=637302516704398374',
 'o_price': '49800',
 's_price': '34800',
 'title': '[맥심] 맥심 모카골드 커피믹스 320T +썸머 비치타월 증정 '}
2020-07-13 15:41:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1771780201&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:12 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1274338554&ver=637302516704398374>
{'discount_rate': '-29.69%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1274338554&ver=637302516704398374',
 'o_price': '130000',
 's_price': '39900',
 'title': '[벨리에르] 인견 토퍼 침구 풀세트(사이즈 3가지) '}
2020-07-13 15:41:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=16

2020-07-13 15:41:14 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1595549283&ver=637302516704398374>
{'discount_rate': '-94.0%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1595549283&ver=637302516704398374',
 'o_price': '22410',
 's_price': '21290',
 'title': '[비치붐] UV 울트라 플랩캡 4종(택1) 성인플랩캡/ 비치붐 '}
2020-07-13 15:41:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1669313542&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:14 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1430266775&ver=637302516704398374>
{'discount_rate': '-29.0%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1430266775&ver=637302516704398374',
 'o_price': '43000',
 's_price': '12900',
 'title': '시크루즈/12%추가/원피스/바캉스룩/SET상품/빅사이즈 '}
2020-07-13 15:41:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=177

2020-07-13 15:41:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1674065361&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:16 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1494985439&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1674065361&ver=637302516704398374>
{'discount_rate': '-91.57%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1674065361&ver=637302516704398374',
 'o_price': '26900',
 's_price': '24900',
 'title': '[뼈로가는칼슘두유] 검은콩 190ml 72팩/두유 '}
2020-07-13 15:41:16 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1494985439&ver=637302516704398374>
{'discount_rate': '-69.34%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1494985439&ver=637302516704398374',
 'o_pric

2020-07-13 15:41:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1667572441&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:17 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1686036816&ver=637302516704398374>
{'discount_rate': '-88.47%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1686036816&ver=637302516704398374',
 'o_price': '26600',
 's_price': '23800',
 'title': '[제주삼다수] 제주삼다수 2L 24병/생수전문배송 '}
2020-07-13 15:41:17 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1667572441&ver=637302516704398374>
{'discount_rate': '-92.16%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1667572441&ver=637302516704398374',
 'o_price': '59900',
 's_price': '55800',
 'title': '[피코크] 피코크 특제육수 조선호텔 김치 9kg/box '}
2020-07-13 15:41:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=15773633

...

2020-07-13 15:41:27 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=176368626&ver=637302516704398374>
{'discount_rate': '-29.1%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=176368626&ver=637302516704398374',
 'o_price': '19600',
 's_price': '5900',
 'title': '[란제리아] 여성팬티5매 여자 미디 맥시 속옷 삼각 위생 면 스판 '}
2020-07-13 15:41:27 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1770086331&ver=637302516704398374>
{'discount_rate': '-49.0%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1770086331&ver=637302516704398374',
 'o_price': '39000',
 's_price': '19500',
 'title': '[웨스트우드] 웨스트우드 SUMMER 바람막이/냉감티/반바지 외 특가전 '}
2020-07-13 15:41:27 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=355494519&ver=637302516704398374>
{'discount_rate': '-29.0%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=355494519&ver=637302516704398374',
 'o_price': '33000',
 's_price': '9900',
 

2020-07-13 15:41:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1771533911&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:29 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1635187949&ver=637302516704398374>
{'discount_rate': '-65.67%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1635187949&ver=637302516704398374',
 'o_price': '25350',
 's_price': '16900',
 'title': '메론 머스크 멜론  특대과 2수 4kg '}
2020-07-13 15:41:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1581237483&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:29 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1514517550&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:29 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item

2020-07-13 15:41:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1831976647&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=816472004&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=153279336&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=699501445&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:31 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1831976647&ver=637302516704398374>
{'discount_rate': '-38.93%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=

2020-07-13 15:41:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=1843766339&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://item.gmarket.co.kr/Item?goodscode=920819639&ver=637302516704398374> (referer: http://corners.gmarket.co.kr/Bestsellers)
2020-07-13 15:41:32 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1843766339&ver=637302516704398374>
{'discount_rate': '-99.0%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1843766339&ver=637302516704398374',
 'o_price': '70900',
 's_price': '70900',
 'title': '[크리넥스] 스타일마스크 블랙 대형 3PX30개/총 90개입/마스크 '}
2020-07-13 15:41:32 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=920819639&ver=637302516704398374>
{'discount_rate': '-68.87%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=920819639&ver=637302516704398374',
 '

2020-07-13 15:41:34 [scrapy.core.scraper] DEBUG: Scraped from <200 http://item.gmarket.co.kr/Item?goodscode=1219336883&ver=637302516704398374>
{'discount_rate': '-35.45%',
 'link': 'http://item.gmarket.co.kr/Item?goodscode=1219336883&ver=637302516704398374',
 'o_price': '29900',
 's_price': '10900',
 'title': '[밀크바오밥] 1+1 퍼퓸 샴푸/트리트먼트/바디워시/바디로션 500ml '}
2020-07-13 15:41:34 [scrapy.core.engine] INFO: Closing spider (finished)
2020-07-13 15:41:34 [scrapy.extensions.feedexport] INFO: Stored csv feed (200 items) in: GmarketBestsellers.csv
2020-07-13 15:41:34 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 75127,
 'downloader/request_count': 203,
 'downloader/request_method_count/GET': 203,
 'downloader/response_bytes': 7684618,
 'downloader/response_count': 203,
 'downloader/response_status_count/200': 203,
 'elapsed_time_seconds': 24.17128,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2020, 7, 13, 6, 41, 34, 573136),
 'item_scraped_count

In [21]:
!ls crawler/

GmarketBestsellers.csv crawler                scrapy.cfg


In [26]:
df = pd.read_csv("GmarketBestsellers.csv")

In [28]:
df.tail()

,discount_rate,link,o_price,s_price,title
195,-38.92%,http://item.gmarket.co.kr/Item?goodscode=14819...,24800,9900,쫀득쫀득 혼합찹쌀떡(40gX21개) 1+1+1
196,-38.94%,http://item.gmarket.co.kr/Item?goodscode=17880...,34800,13900,밥상마루 100% 참기름 350ml + 들기름 350ml
197,-29.72%,http://item.gmarket.co.kr/Item?goodscode=14963...,31900,9800,[니베아] 니베아 데오드란트 스프레이X2개 / 데오도란트
198,-38.93%,http://item.gmarket.co.kr/Item?goodscode=14598...,29800,11900,[동서] 오레오웨하스150g x3개입+간식쟁반
199,-35.45%,http://item.gmarket.co.kr/Item?goodscode=12193...,29900,10900,[밀크바오밥] 1+1 퍼퓸 샴푸/트리트먼트/바디워시/바디로션 500ml
